### MLP with Tensorboard 3

see:  https://github.com/nlintz/TensorFlow-Tutorials/blob/master/9_tensorboard.py

testing optimization loss function and gradient clipping

https://github.com/tensorflow/tensorflow/blob/c936c06a20b867eaa868e9f2c4fb53856391b18f/tensorflow/contrib/layers/python/layers/optimizers.py#L114

https://github.com/kapild/deeplearning/blob/8adb8452a8b16e15bdcec23153a78a3b1fa02f76/tensorflow/contrib/layers/python/layers/optimizers_test.py

In [1]:
# A Multilayer Perceptron implementation example using TensorFlow library.
# This example is using the MNIST database of handwritten digits
# (http://yann.lecun.com/exdb/mnist/)=


# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import tensorflow as tf
import numpy as np
import input_data

In [3]:
%run infimnist.py

In [4]:
# Parameters
num_epochs = 5000
batch_size = 125

learning_rate = 0.001

In [5]:
# Network Parameters
n_hidden_1 = 1000 # 1st layer num features
n_hidden_2 = 500 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

std_0 = 1.0/np.sqrt(n_input)
std_h1 = 1.0/np.sqrt(n_hidden_1)
std_h2 = 1.0/np.sqrt(n_hidden_2)


In [6]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [7]:
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [8]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=std_0)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=std_h1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=std_h2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev=0.01)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.001))
}

In [9]:
# Construct model
pred = multilayer_perceptron(x, weights, biases )

In [10]:
cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
gs = tf.get_variable("global_step",[],trainable=False,initializer=tf.constant_initializer(0))
lr = tf.constant(learning_rate) 

In [11]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

In [12]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import logging_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import variables as vars_
from tensorflow.python.training import training as train

In [13]:
# we can check to see if we built the model sanely

In [14]:
variables = vars_.trainable_variables()
for v in variables:
    print v.name

Variable:0
Variable_1:0
Variable_2:0
Variable_3:0
Variable_4:0
Variable_5:0


In [15]:
train_op = tf.contrib.layers.optimize_loss(cost, global_step=gs, learning_rate=lr,optimizer="RMSProp")

In [16]:
#with tf.name_scope("training accuracy"):
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) # Count correct predictions
acc_op = tf.reduce_mean(tf.cast(correct_prediction, "float"))  # Cast boolean to float to average
tf.scalar_summary("accuracy", acc_op)

<tf.Tensor 'ScalarSummary_2:0' shape=() dtype=string>

In [17]:
infiminst = InfiMNIST()

In [21]:
!rm -rf ./logs/nn_logs

In [22]:
!ls logs

nn_logs_3


In [ ]:
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.train.SummaryWriter("./logs/nn_logs", sess.graph) # for 0.8
    merged = tf.merge_all_summaries()

    tf.initialize_all_variables().run()

    for epoch in range(num_epochs):
        infiminst.next_epoch()
        mnist = input_data.read_data_sets(".", one_hot=True)
        
        trX, trY = mnist.train.images, mnist.train.labels
        for start, end in zip(range(0, len(trX), batch_size), range(125, len(trX), batch_size)):
            sess.run(train_op, feed_dict={x: trX[start:end], y: trY[start:end]})
            
        teX, teY = mnist.test.images, mnist.test.labels
        summary, acc = sess.run([merged, acc_op], feed_dict={x: teX, y: teY})
        writer.add_summary(summary, epoch)  
        print(epoch, acc)
        writer.flush()

(0, 0.95639998)
(1, 0.97289997)
(2, 0.96950001)
(3, 0.98049998)
(4, 0.97659999)
(5, 0.97850001)
(6, 0.97860003)
(7, 0.98199999)
(8, 0.98460001)
(9, 0.98449999)
(10, 0.98079997)
(11, 0.98290002)
(12, 0.97960001)
(13, 0.98549998)
(14, 0.9835)
(15, 0.98559999)
(16, 0.98519999)
(17, 0.98519999)
(18, 0.98559999)
(19, 0.98119998)
(20, 0.9853)
(21, 0.98680001)
(22, 0.9853)
(23, 0.98570001)
(24, 0.98250002)
(25, 0.98509997)
(26, 0.9849)
(27, 0.9853)
(28, 0.98640001)
(29, 0.98519999)
(30, 0.9874)
(31, 0.98820001)
(32, 0.98680001)
(33, 0.98580003)
(34, 0.98689997)
(35, 0.98379999)
(36, 0.98519999)
(37, 0.98259997)
(38, 0.98460001)
(39, 0.98210001)
(40, 0.98430002)
(41, 0.98519999)
(42, 0.98610002)
(43, 0.98869997)
(44, 0.99019998)
(45, 0.98769999)
(46, 0.98400003)
(47, 0.98400003)
(48, 0.9849)
(49, 0.97820002)
(50, 0.9842)
(51, 0.98799998)
(52, 0.98729998)
(53, 0.98360002)
(54, 0.98580003)
(55, 0.98580003)
(56, 0.98839998)
(57, 0.98659998)
(58, 0.98580003)
(59, 0.98830003)
(60, 0.98809999)
(61, 